# Neeps- Medium

## 6.
**Show the 'size' of each of the co72010 events. Size is the total number of students attending each event.**

## 7.
**For each post-graduate module, show the size of the teaching team. (post graduate modules start with the code co7).**

## 8.
**Give the full name of those modules which include events taught for fewer than 10 weeks.**

## 9.
**Identify those events which start at the same time as one of the co72010 lectures.**

## 10.
**How many members of staff have contact time which is greater than the average?**